# Smart Grocery Cart Assistant  
*Built on top of the JioMart Retail Product Catalog, [sourced](https://www.kaggle.com/datasets/satyamsundaram/jiomart-products-dataset) from Kaggle*

---

## Objective  
Create a Gradio-based AI assistant that recommends a weekly grocery shopping cart based on the user's dietary needs and preferences. The app uses Retrieval-Augmented Generation (RAG) on a structured JioMart product dataset, generating explainable, structured results that are visually rendered like a real shopping cart.

---

## Part 1 — User App (Essential Features)

### Inputs
- **Grocery Needs**: Free-form text input  
  *(e.g., “high protein, no besan or curd”)*
  
### Output
- **Suggestion**: A natural language explanation of what's recommended and why
- **Shopping Cart**: Structured as a visual gallery containing:
  - Product Name
  - Quantity
  - Price
  - Product Image

---

## Part 2 — Visual Experience

### Visual Cart (Gradio `gr.Gallery`)
The cart is rendered in a grid format using `gr.Gallery`, where each item includes:
- Product image (`image_url`)
- Caption text with:
  - `item_name`
  - `Qty.{quantity}`
  - `₹total_price`

This provides a realistic and user-friendly shopping experience.

---

## Part 3 — Developer-Facing Advanced Settings

Shown under a collapsible **LLM Settings (Advanced)** section in the UI.

### Model & Generation Settings
- **Model Selector**:
  - GPT-4o-Mini (`gpt-4o-mini`)
  - LLaMA 3.3 70B (`llama-3.3-70b-versatile`)
- **Temperature Slider**: Adjustable between 0.0 and 1.5

These settings allow developers to tune how deterministic or creative the model’s responses are.

---

## Backend Setup

### Vector Store and RAG
- **Embedding Model**: LLaMA 3.2 (3B) via Ollama
- **Vector Database**: ChromaDB
- **RAG Workflow**:
  1. Load product catalog using `CSVLoader`
  2. Parse and clean fields into `Document` objects with metadata
  3. Generate embeddings and index documents
  4. Perform similarity search based on user preferences
  5. Use an LLM to generate structured output parsed via `PydanticOutputParser`

---

## Sample CSV Schema Mapping

| Field in CSV     | Mapped Use              |
|------------------|-------------------------|
| `title`          | `item_name` and `quantity` (parsed from name) |
| `discountedPrice`| `unit_price`            |
| `filename`       | `image_url`             |
| `subType`        | `sub_category`          |
| `type`           | `category`              |


## Setup

In [2]:
# !uv pip install langchain_google_vertexai langchain_deepseek langchain_anthropic langchain_openai langchain_groq

In [1]:
import gradio as gr
import pandas as pd
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings
from langchain_community.document_loaders import CSVLoader
from langchain.chat_models import init_chat_model
from langchain_openai import OpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain.schema.output_parser import OutputParserException
from langchain_core.documents import Document
from pydantic import BaseModel
import os
from dotenv import load_dotenv

In [2]:
load_dotenv(override=True)

True

In [3]:
#Check for API Keys
api_keys = ["OPENAI_API_KEY","GROQ_API_KEY"]
for api_key in api_keys:
    if api_key not in os.environ:
        os.environ[api_key] = getpass.getpass("Enter the API key: ")

## Loading and Preprocessing Data

In [4]:
# Load and preprocess data
loader = CSVLoader(file_path="jiomart_products_database.csv", source_column="title")
documents_raw = loader.load()

In [5]:
# Convert page_content string to dict and build metadata
documents = []
for doc in documents_raw:
    try:
        row_data = dict(
            line.split(":", 1) for line in doc.page_content.split("\n") if ":" in line
        )
        row_data = {k.strip(): v.strip() for k, v in row_data.items()}

        page_text = f"Name: {row_data.get('title', '')} | Sub-type: {row_data.get('subType', '')} | Type: {row_data.get('type', '')} | Price: {row_data.get('discountedPrice', 0)} | Image: {row_data.get('filename', '')}"
        metadata = {
            "category": row_data.get("type", ""),
            "sub_category": row_data.get("subType", "")
        }

        documents.append(Document(page_content=page_text, metadata=metadata))

    except Exception as e:
        print("Skipping row due to error:", e)

In [6]:
len(documents)

5672

## Storing in Vector DB

In [7]:
# Create embeddings
!ollama pull llama3.2:1b
embedding_model = OllamaEmbeddings(model="llama3.2:1b")

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest 
pulling 74701a8c35f6: 100% ▕██████████████████▏ 1.3 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da: 100% ▕██████████████████▏ 7.7 KB                         
pulling a70ff7e570d9: 100% ▕██████████████████▏ 6.0 KB                         
pulling 4f659a1e86d7: 100% ▕██████████████████▏  485 B                         
verifying sha256 digest 
writing manifest 
success 


In [8]:
# Store in Chroma - 100 rows only for quicker processing
vectorstore = Chroma.from_documents(documents[:100], embedding_model)
retriever = vectorstore.as_retriever()

## RAG Pipeline

In [9]:
# Pydantic schema
class GroceryItem(BaseModel):
    item_name: str
    price: float
    quantity: int
    image_url: str

class GroceryOutput(BaseModel):
    reasoning: str
    items: list[GroceryItem]

parser = PydanticOutputParser(pydantic_object=GroceryOutput)

In [10]:
# Prompt Template
template = ChatPromptTemplate.from_template("""
You are a helpful grocery assistant. Based on the user preferences below and the product catalog context, recommend a short list of minimum 10 items for a weekly diet intake. 
Make sure items are not repeated in the output.

User Preferences: {preferences}

<context>
{context}
</context>

Return your response as a JSON following this format:
{format_instructions}
""")

In [11]:
#Model name map
model_name_map = {
    "GPT-4o-Mini": "gpt-4o-mini",
    "Llama-3.3": "llama-3.3-70b-versatile"
}

model_choices = [
    "GPT-4o-Mini (OpenAI)",
    "Llama-3.3 (70B) (Groq)",
]

In [12]:
# Model Selector
def get_llm(model_choice, temperature):
    if "Groq" in model_choice:
        return init_chat_model(model_name_map[model_choice.split(" ")[0]], model_provider="groq", temperature = temperature)
    elif "OpenAI" in model_choice:
        return init_chat_model(model_name_map[model_choice.split(" ")[0]], model_provider="openai", temperature = temperature)
    else:
        raise ValueError("Invalid model")

In [ ]:
#Testing RAG pipeline line-by-line
preference = "I want a high fat diet"
context_docs = retriever.invoke(preference) #Retrieving relevant food items
relevant_text = "\n".join([doc.page_content for doc in context_docs])
llm = get_llm(model_choices[0], 0)
chain = template | llm | parser
input_dict = {
    "preferences": preference,
    "context": relevant_text,
    "format_instructions": parser.get_format_instructions()
}
chain.invoke(input_dict).model_dump()


In [ ]:
# RAG pipeline
def generate_cart(model_choice, user_input):
    context_docs = retriever.invoke(user_input["preferences"]) #Retrieving relevant food items
    relevant_text = "\n".join([doc.page_content for doc in context_docs]) #Joining the food items
    llm = get_llm(user_input["model_choice"], user_input["temperature"])
    
    # #Invoking - Option 1
    # prompt = template.format(
    #     preferences=user_input["preferences"],
    #     context=relevant_text,
    #     format_instructions=parser.get_format_instructions()
    # )
    # output = llm.invoke(prompt)

    #Invoking - Option 2
    input_dict = {
        "preferences": user_input["preferences"],
        "context": relevant_text,
        "format_instructions": parser.get_format_instructions()
    }
    chain = template | llm
    output = chain.invoke(input_dict)

    try:
        result = parser.parse(output.content)
    except OutputParserException:
        return "Could not parse output.", None
    return result

## User Interface

### Simple UI

In [ ]:
def gradio_interface(preferences, model_choice, temperature):
    user_input = {
        "preferences": preferences,
        "model_choice": model_choice,
        "temperature": temperature,
    }
    result = generate_cart(model_choice, user_input)
    if not result or isinstance(result, str):
        return result, None
    if isinstance(result, tuple):
        explanation, _ = result
        return explanation, None

    # Build a gallery format: [(image_url, caption), ...]
    gallery_items = [
        (item.image_url, f"{item.item_name}\nQty.{item.quantity}\n₹{item.quantity*item.price}\n")
        for item in result.items
    ]
    return result.reasoning, gallery_items


demo = gr.Interface(
    fn=gradio_interface,
    inputs=[
        gr.Textbox(label="Describe your grocery needs (e.g., 'high protein, no besan or curd')"),
        gr.Dropdown(label="Model", choices=model_choices),
        gr.Slider(minimum=0.0, maximum=1.5, value=0.7, step=0.1, label="Temperature")
    ],
    outputs=[
        gr.Textbox(label="Considerations"),
        gr.Gallery(label="Shopping Cart", columns=3, height="auto")
    ],
    title="Smart Grocery Cart Assistant",
    description="Get a product list tailored to your dietary preferences."
)


In [ ]:
if __name__ == "__main__":
    demo.launch()

### Better UI

In [ ]:
def gradio_interface(preferences, model_choice, temperature):
    user_input = {
        "preferences": preferences,
        "model_choice": model_choice,
        "temperature": temperature,
    }
    result = generate_cart(model_choice, user_input)
    if not result or isinstance(result, str):
        return result, []
    if isinstance(result, tuple):
        explanation, _ = result
        return explanation, []
    gallery_items = [
        (item.image_url, f"{item.item_name}\nQty.{item.quantity}\n₹{item.quantity*item.price}\n")
        for item in result.items
    ]
    return result.reasoning, gallery_items

In [ ]:
# Gradio Blocks layout
with gr.Blocks(title="Smart Grocery Cart Assistant") as demo:
    gr.Markdown("## Smart Grocery Cart Assistant")
    gr.Markdown("Enter your grocery preferences and let AI suggest a weekly cart!")

    with gr.Row():
        preferences_input = gr.Textbox(
            label="Describe your grocery needs",
            placeholder="e.g., high protein, no besan or curd",
            lines=2
        )

    with gr.Accordion("LLM Settings (Advanced)", open=False):
        model_dropdown = gr.Dropdown(
            label="Model",
            choices=model_choices,
            value=model_choices[0]
        )
        temperature_slider = gr.Slider(
            minimum=0.0, maximum=1.5, value=0.7, step=0.1,
            label="Temperature"
        )

    run_button = gr.Button("Generate Cart")

    with gr.Row():
        reasoning_output = gr.Textbox(label="Remarks", lines=2)

    with gr.Row():
        cart_output = gr.Gallery(label="Shopping Cart", columns=3, height="auto")

    run_button.click(
        fn=gradio_interface,
        inputs=[preferences_input, model_dropdown, temperature_slider],
        outputs=[reasoning_output, cart_output]
    )

In [ ]:
demo.launch(inline=True)